In [43]:
# !pip install requests
import requests
import pandas as pd

In [44]:
# User specific API key so will need changing...

date_params = ["start=1990&end=2000","start=2000&end=2010","start=2010",]
# date_params = ["start=1990"]
urls_all = [f"https://api.eia.gov/v2/international/data/?api_key=KB7o3u8fD9dPM6fSnjaG0cBaBBaQ7qyW68Vi52PM&frequency=annual&data[0]=value"
        "&facets[activityId][]=1&facets[productId][]=7&facets[unit][]=MT&facets[countryRegionTypeId][]=c"
        f"&{d}&sort[0][column]=countryRegionId&sort[0][direction]=asc&offset=0&length=5000" #my API key
        for d in date_params
        ]
urls_met = [f"https://api.eia.gov/v2/international/data/?api_key=KB7o3u8fD9dPM6fSnjaG0cBaBBaQ7qyW68Vi52PM&frequency=annual&data[0]=value"
        "&facets[activityId][]=1&facets[productId][]=130&facets[unit][]=MT&facets[countryRegionTypeId][]=c"
        f"&{d}&sort[0][column]=countryRegionId&sort[0][direction]=asc&offset=0&length=5000" #my API key
        for d in date_params
        ]

In [45]:
dfs_all = []
for url in urls_all:
    response = requests.get(url)
    response.raise_for_status()
    df = pd.DataFrame(response.json()["response"]["data"])
    dfs_all.append(df)
data_all = pd.concat(dfs_all)

dfs_met = []
for url in urls_met:
    response = requests.get(url)
    response.raise_for_status()
    df = pd.DataFrame(response.json()["response"]["data"])
    dfs_met.append(df)
data_met = pd.concat(dfs_met)

In [46]:
def get_data(data):
    # Make sure your 'value' column is numeric
    data['value'] = pd.to_numeric(data['value'], errors='coerce')

    # Pivot the DataFrame
    pivot_df = data.pivot_table(
        index=["countryRegionId", "period"],
        columns="productName",
        values="value",
        fill_value=0
    ).reset_index()
    return pivot_df

In [47]:
all_coal = get_data(data_all)
met_coal = get_data(data_met)

coal_df = all_coal[['countryRegionId', 'period', 'Coal']]
met_df  = met_coal[['countryRegionId', 'period', 'Metallurgical coal']]

combined = coal_df.merge(
    met_df,
    on=['countryRegionId', 'period'],
    how='outer'
)

combined = combined.sort_values(['countryRegionId', 'period'])

In [48]:
# Rename columns
combined = combined.rename(columns={
    "countryRegionId": "country code",
    "period": "year",
    "Coal": "total coal (MT)",
    "Metallurgical coal": "metallurgical coal (MT)"
})

combined['total coal (MT)'] = combined['total coal (MT)'] / 1000
combined['metallurgical coal (MT)'] = combined['metallurgical coal (MT)'] / 1000

In [49]:
combined.to_csv('eia_production.csv', index=False)